In [ ]:
import gradio as gr
import requests
import base64
import re
import azure.cognitiveservices.speech as speechsdk

# 🔐 API 키 및 엔드포인트 설정
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = ""
openai_rag_key = ""

speech_key = ""
speech_region = "eastus2"

# 🖼️ 이미지 → base64 변환
def image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# 🧹 특수문자 정리
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 🗣️ 음성 생성 (SSML + 속도 조절 + 음성 선택)
def speak_text(text, speed="0%", voice_name="ko-KR-HyunsuMultilingualNeural"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="{voice_name}">
        <prosody rate=\"{speed}\">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 👁️ GPT Vision OCR
def get_vision_ocr(base64_image):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사 필기 자료입니다. 이미지에 포함된 텍스트를 문맥에 맞게 정리하여 글을 작성하세요. "
                "그리고 추출된 내용 외에도 관련된 역사적 사건, 시대적 배경, 인물, 제도, 흐름 등을 보완하여 설명하세요. "
                "한국사능력검정시험을 준비하는 수험생에게 도움이 되는 형태로 서술해 주세요."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {"type": "text", "text": "이미지의 텍스트를 바탕으로 전체 맥락을 풍부하게 설명해주세요."}
            ]
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.95,
        "max_tokens": 4096
    }
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 💡 RAG 기반 GPT 요약
def get_rag_answer(input_text, target_tokens=500, target_chars=300):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. 주어진 지문을 요약하고 역사적 배경과 키워드를 설명하세요. "
                f"요약은 반드시 {target_chars}자 이내로 작성하세요. "
                "가능하다면 해당 제한에 최대한 가까운 길이로 작성해 주세요. 줄글 형식으로 서술하십시오."
            )
        },
        {
            "role": "user",
            "content": f"다음은 필기 이미지로부터 생성된 지문입니다:\n\"\"\"\n{input_text}\n\"\"\""
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": target_tokens,
        "frequency_penalty": 0.25
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 🎯 전체 파이프라인
def process_image_pipeline(image, speed_label, char_label, voice_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    voice_map = {
        "👩‍🦰 여자1": "ko-KR-SunHiNeural",
        "👩 여자2": "ko-KR-SoonBokNeural",
        "👨 남자1": "ko-KR-InJoonNeural",
        "👨‍🦱 남자2": "ko-KR-HyunsuMultilingualNeural"
    }
    char_token_map = {
        "약 500자 요약 (1분 예상)": (500, 300),
        "약 1000자 요약 (3분 예상)": (1000, 800),
        "약 1500자 요약 (5분 예상)": (2500, 2100)
    }
    # 속도, 음성, 요약 길이 설정
    rate = speed_map[speed_label]
    voice_name = voice_map[voice_label]
    target_chars, target_tokens = char_token_map[char_label]

    # 그라디오에 사용되게 이미지 처리
    base64_image = image_to_base64(image) if image else None

    # GPT Vision OCR
    vision_summary = get_vision_ocr(base64_image)
    # GPT RAG 요약
    rag_answer = get_rag_answer(vision_summary, target_tokens=target_tokens, target_chars=target_chars)

    # 특수문자없애기
    rag_output_text = clean_special_characters(rag_answer)
    vision_summary_text = clean_special_characters(vision_summary)

    # 음성생성
    audio = speak_text(rag_output_text, rate, voice_name)

    # rag_output_text = f"[총 {len(cleaned)}자] " + cleaned
    # vision_summary_text = f"[총 {len(cleaned_v)}자] " + cleaned_v
    return vision_summary_text, rag_output_text, audio

# 🖼️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → GPT 요약 → 음성 출력")

    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            chars = gr.Radio(
                ["약 500자 요약 (1분 예상)", "약 1000자 요약 (3분 예상)", "약 1500자 요약 (5분 예상)"],
                label="요약 길이 (글자 수 기준)",
                value="약 500자 요약 (1분 예상)"
            )
            voice = gr.Radio(
                ["👩‍🦰 여자1", "👩 여자2", "👨 남자1", "👨‍🦱 남자2"],
                label="TTS 음성 선택",
                value="👨‍🦱 남자2"
            )
            submit = gr.Button("응답 받기")
        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision OCR", lines=3)
            rag_output = gr.Textbox(label="2️⃣ 요약 결과", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)

    submit.click(
        fn=process_image_pipeline,
        inputs=[image, speed, chars, voice],
        outputs=[vision_output, rag_output, audio]
    )

demo.launch()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.


In [ ]:
import gradio as gr
import requests
import base64
import re
import azure.cognitiveservices.speech as speechsdk

# 🔐 API 키 및 엔드포인트 설정
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = "https://6b013-azure-ai-service.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
openai_rag_key = ""

speech_key = ""
speech_region = "eastus2"

# 🖼️ 이미지 → base64 변환
def image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# 🧹 특수문자 정리
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 🗣️ 음성 생성 (SSML + 속도 조절 + 음성 선택)
def speak_text(text, speed="0%", voice_name="ko-KR-HyunsuMultilingualNeural"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="{voice_name}">
        <prosody rate=\"{speed}\">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 👁️ GPT Vision OCR
def get_vision_ocr(base64_image):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사 필기 자료입니다. 이미지에 포함된 텍스트를 문맥에 맞게 정리하여 글을 작성하세요. "
                "그리고 추출된 내용 외에도 관련된 역사적 사건, 시대적 배경, 인물, 제도, 흐름 등을 보완하여 설명하세요. "
                "한국사능력검정시험을 준비하는 수험생에게 도움이 되는 형태로 서술해 주세요."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {"type": "text", "text": "이미지의 텍스트를 바탕으로 전체 맥락을 풍부하게 설명해주세요."}
            ]
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.95,
        "max_tokens": 4096
    }
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 💡 RAG 기반 GPT 요약
def get_rag_answer(input_text, target_tokens=500, target_chars=300):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. 주어진 지문을 요약하고 역사적 배경과 키워드를 설명하세요. "
                f"요약은 반드시 {target_chars}자 이내로 작성하세요. "
                "가능하다면 해당 제한에 최대한 가까운 길이로 작성해 주세요. 줄글 형식으로 서술하십시오."
            )
        },
        {
            "role": "user",
            "content": f"다음은 필기 이미지로부터 생성된 지문입니다:\n\"\"\"\n{input_text}\n\"\"\""
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": target_tokens,
        "frequency_penalty": 0.25
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 🎯 전체 파이프라인
def process_image_pipeline(image, speed_label, char_label, voice_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    voice_map = {
        "👩‍🦰 여자1": "ko-KR-SunHiNeural",
        "👩 여자2": "ko-KR-SoonBokNeural",
        "👨 남자1": "ko-KR-InJoonNeural",
        "👨‍🦱 남자2": "ko-KR-HyunsuMultilingualNeural"
    }
    char_token_map = {
        "약 500자 요약 (1분 예상)": (500, 300),
        "약 1000자 요약 (3분 예상)": (1000, 800),
        "약 1500자 요약 (5분 예상)": (2500, 2100)
    }
    # 속도, 음성, 요약 길이 설정
    rate = speed_map[speed_label]
    voice_name = voice_map[voice_label]
    target_chars, target_tokens = char_token_map[char_label]

    # 그라디오에 사용되게 이미지 처리
    base64_image = image_to_base64(image) if image else None

    # GPT Vision OCR
    vision_summary = get_vision_ocr(base64_image)
    # GPT RAG 요약
    rag_answer = get_rag_answer(vision_summary, target_tokens=target_tokens, target_chars=target_chars)

    # 특수문자없애기
    rag_output_text = clean_special_characters(rag_answer)

    # 음성생성
    audio = speak_text(rag_output_text, rate, voice_name)

    # rag_output_text = f"[총 {len(cleaned)}자] " + cleaned
    # vision_summary_text = f"[총 {len(cleaned_v)}자] " + cleaned_v
    return rag_output_text, audio

# 🖼️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → GPT 요약 → 음성 출력")

    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            chars = gr.Radio(
                ["약 500자 요약 (1분 예상)", "약 1000자 요약 (3분 예상)", "약 1500자 요약 (5분 예상)"],
                label="요약 길이 (글자 수 기준)",
                value="약 500자 요약 (1분 예상)"
            )
            voice = gr.Radio(
                ["👩‍🦰 여자1", "👩 여자2", "👨 남자1", "👨‍🦱 남자2"],
                label="TTS 음성 선택",
                value="👨‍🦱 남자2"
            )
            submit = gr.Button("응답 받기")
        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision OCR", lines=3)
            rag_output = gr.Textbox(label="2️⃣ 요약 결과", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)

    submit.click(
        fn=process_image_pipeline,
        inputs=[image, speed, chars, voice],
        outputs=[vision_output, rag_output, audio]
    )

demo.launch()